# Polars Workshop

PyCon 2025

Matt Harrison

In [ ]:
import polars as pl 
pl.__version__

## Getting Data

If you threaten to delete your Strava account, they will let you download all your data [^1]

[^1]: https://support.strava.com/hc/en-us/articles/216918437-Exporting-your-Data-and-Bulk-Export

In [ ]:
import polars as pl
import gpxpy
import numpy as np

def gpx_to_polars(fname):
    # Parse the GPX file
    data = gpxpy.parse(open(fname))
    prev = None
    data_dict = {'course': [],
                 'distance_2d': [],
                 'latitude': [],
                 'longitude': [],
                 'time': [],
                 'elevation': [],
                 'speed_between': [],
                }
    name = None
    type_ = None
    filename = fname
    
    # Iterate through tracks, segments, and points
    for track in data.tracks:
        if name is None:
            name = track.name
            type_ = track.type

        for seg in track.segments:
            for i, pt in enumerate(seg.points):
                if prev is None:
                    prev = pt
                for key in data_dict:
                    attr = getattr(pt, key)
                    if callable(attr):
                        data_dict[key].append(attr(prev))
                    else:
                        data_dict[key].append(attr)
                prev = pt
    data_dict['name'] = [name] * len(data_dict['course'])
    data_dict['type'] = [type_] * len(data_dict['course'])
    data_dict['filename'] = [filename] * len(data_dict['course'])
    # Create a Polars DataFrame
    df = (pl.DataFrame(data_dict)
    )
    return df
    (df
        #.with_columns([pl.col("time").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%SZ", strict=False)])
        .with_columns(
            travelled= pl.col("distance_2d").cum_sum(),
            elapsed=(pl.col("time") - pl.col("time").min()).dt.total_seconds()
        )
        .with_columns(        
            avg_velocity=pl.col("travelled") / pl.col("elapsed"),
            rolling_travelled=pl.col("travelled").rolling_mean(window_size=5),
            rolling_elapsed=pl.col("elapsed").rolling_mean(window_size=5),
        )
        .with_columns(
            rolling_velocity=pl.col("rolling_travelled") / pl.col("rolling_elapsed"),
            rolling_between=pl.col("speed_between").rolling_mean(window_size=5),
        )
    )

    return df

# Example usage:
#crash_gpx = gpxpy.parse(open('/Users/matt/Downloads/Face_plant.gpx'))




## Load Data

In [ ]:
import polars as pl
import polars.selectors as cs
#raw = pl.read_parquet('data/activities.parquet')
raw = pl.read_parquet('data/bikeski.parquet')
print(raw)

### Exercise

- Load the data above.
- Inspect the `.shape` attribute of the data

## Types

In [ ]:
print(raw.columns)

In [ ]:
cols = ['course', 'distance_2d', 'latitude', 'longitude', 'time', 'elevation',
         'speed_between', 'name', 'type', 'filename']
raw[cols].dtypes

In [ ]:
# 345 MB
raw[cols].estimated_size()

## Ints

In [ ]:
# add a id from the filename
print(raw
 .select(cols)
 .with_columns(id=pl.col('filename')
               .str.split('/')
               .list.get(-1)
               .str.replace('.gpx', '')
               .cast(pl.Int64)
               )
)

In [ ]:
# add an elapsed column to each activity
# add a id from the filename

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )

(raw
 .select(cols)
 .with_columns(id=calc_id)
)

In [ ]:
# add an elapsed column to each activity
# add a id from the filename

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed)
)

In [ ]:
# cast ints to int16
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int16))
)

In [ ]:
# describe

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed)
 .select(cs.integer())
 .describe()
)

In [ ]:
import numpy as np
np.iinfo(np.int16)

In [ ]:
import numpy as np
np.iinfo(np.int32)

In [ ]:
# cast ints to int32

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))

)

### Execise: Memory Ints

- Determine how much memory is used when casting *elapsed* to an Int32

## Floats


In [ ]:
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))
 .select(cs.float())
)

In [ ]:
# cast
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))
 .select(cs.float())
 .columns
)

### Exercise:

- How much memory is used if we cast the float columns to Float32?

## Strings

In [ ]:

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))
 .with_columns(pl.col(float_cols).cast(pl.Float32))
 .select(cs.string())
)

In [ ]:
pl.col('').str

In [ ]:
dir(pl.col('').str)

In [ ]:
# cast to categorical removes 50% of the size
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))
 .with_columns(pl.col(float_cols).cast(pl.Float32))
 .with_columns(cs.string().cast(pl.Categorical))
 .estimated_size()
)

In [ ]:
# cast to categorical removes 50% of the size
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']

(raw
 .select(cols)
 # add elapsed time
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(elapsed=calc_elapsed.cast(pl.Int32))
 .with_columns(pl.col(float_cols).cast(pl.Float32))
 #.with_columns(cs.string().cast(pl.Categorical))
 .select(cs.string())
 .columns
)

In [ ]:
# cast to categorical removes 50% of the size
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
 )
)

In [ ]:
# explore categorical capabilities
pl.col('cat').cat

In [ ]:
dir(pl.col('cat').cat)

### Exercise: Strings

- Write an expression to capitalize the *name* column
- Write an expression to pull off just the filename portion of the *filename* column
- Write an expression to make a string column, *act_id*, that is the IDSHORT_NAMESHORT where
 `IDSHORT` is the first 4 characters of the ID and `NAMESHORT` is the firt 4 characters of the name
- Update the *act_id* expression to convert it from a string to a category.

## Dates



In [ ]:
# convert date to local time
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
)

In [ ]:
# calculate period (a time delta)
# vs elapsed which is numeric seconds
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
 .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))

)

In [ ]:
pl.col('time').dt

In [ ]:
dir(pl.col('time').dt)

### Exercise: Dates

- Write an expression to make a *month* column derived from *time*
- Write an expression to make a *year* column derived from *time*
- Write an expression to make a *utc_time* column derived from *time* with time in UTC

## Missing Data

In [ ]:
# where are values missing (null)?
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
 .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))
 .null_count()
)

In [ ]:
# which rows are they?
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
 .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))
 #.null_count()
 .filter(pl.col('speed_between').is_null())
)

In [ ]:
# what about nans?
calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']

(raw
 .select(cols)
 # add elapsed time
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
 #.null_count()
 # no nan_count
 #.filter(pl.col('speed_between').is_null())
 .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))
 .select(cs.numeric().is_nan().sum())
)

# if then


The Python code:
```
if COND_EXPR:
    EXPR
elif COND_EXPR2:
    EXPR2
...
else:
    EXPRN
```

Is written in Polars as:
```
(pl.when(COND_EXPR)
   .then(EXPR)
.when(COND_EXPR2)
   .then(EXPR2)
... # more when's
.otherwise(EXPRN) # optional
```



In [ ]:
# set speed_between to 0 when distance_2d is 0

(raw
 .select(cols)
 .with_columns(id=calc_id)
 # needs to be in its own with_columns because we reference id
 .with_columns(
    pl.col(float_cols).cast(pl.Float32),
    pl.col(cat_cols).cast(pl.Categorical),
    elapsed=calc_elapsed.cast(pl.Int32),
    time=pl.col('time').dt.convert_time_zone('America/Denver')
 )
 #.null_count()
 # no nan_count
 #.filter(pl.col('speed_between').is_null())
 .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))
 .with_columns(
     speed_between=pl.when(pl.col('speed_between').is_null())
         .then(0)
 )
)


## Create a function

In [ ]:
# note, I'm adding the final columns in the tweak_strava function

import polars as pl
import polars.selectors as cs   

calc_id = (pl.col('filename')
              .str.split('/')
              .list.get(-1)
              .str.replace('.gpx', '')
              .cast(pl.Int64)
             )
calc_elapsed = (
    (pl.col("time") - pl.col("time").min())
    .dt.total_seconds()
    .over(pl.col('id'))
)

float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
cat_cols = ['name', 'type', 'filename']
cols = ['course', 'distance_2d', 'latitude', 'longitude', 'time', 'elevation',
            'speed_between', 'name', 'type', 'filename']
# drop course because it is all null
final_cols = ['distance_2d', 'latitude', 'longitude', 'time', 'elevation', 'speed_between',
    'name', 'type', 'filename', 'id', 'elapsed', 'period']

def tweak_strava(df):
    return (df
        .select(cols) # checking input
        .with_columns(id=calc_id)
        .with_columns(
            pl.col(float_cols).cast(pl.Float32),
            pl.col(cat_cols).cast(pl.Categorical),
            elapsed=calc_elapsed.cast(pl.Int32),
            time=pl.col('time').dt.convert_time_zone('America/Denver')
        )
        .with_columns(period=(pl.col('time')-pl.col('time').first()).over(pl.col('id')))
        .select(final_cols) # checking output
    )

raw = pl.read_parquet('data/bikeski.parquet')
tweak_strava(raw).columns

# Exercise: Top of Notebook

- Move the function cell to the top of the notebook.
- Restart notebook
- Run first cell and ensure that it runs

## Laziness & Chaining

In [ ]:
raw_lazy = pl.scan_parquet('data/bikeski.parquet')
tweak_strava(raw_lazy)

In [ ]:
tweak_strava(raw_lazy).collect()

## Stay in the Sandbox

In [ ]:
# convert elevation from meters to feet
def meters_to_feet(m):
    return m * 3.28084

(tweak_strava(raw)
    .with_columns(elevation_ft=pl.col('elevation').map_elements(meters_to_feet))
    )

In [ ]:
# convert elevation from meters to feet
def meters_to_feet(m):
    return m * 3.28084

(tweak_strava(raw)
    .with_columns(elevation_ft=meters_to_feet(pl.col('elevation')))
    )

In [ ]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
tweaked = (tweak_strava(raw))

In [ ]:
%%timeit
(tweaked
    .with_columns(elevation_ft=pl.col('elevation').map_elements(meters_to_feet))
    )

In [ ]:
# turn warnings back on
warnings.filterwarnings("default")

In [ ]:
%%timeit
(tweaked
    .with_columns(elevation_ft=meters_to_feet(pl.col('elevation')))
    )

In [ ]:
237/8.98

### Exercise: Meters to Miles

- Make a *distance_miles* column that converts the meters in *distance_2d* to miles

## Basics of Plotting

Plotly has basic integration with Altair, but I prefer to use Matplotlib
for static plots.

See my book *Effective Visualization* for more details.

- Histogram
- Line
- Bar
- Scatter

In [ ]:
#distance per activity

def meters_to_feet(m):
    return m * 3.28084

def feet_to_miles(f):
    return f / 5280

def meters_to_miles(m):
    return feet_to_miles(meters_to_feet(m))

(tweak_strava(raw)
    .group_by('name')
    .agg(pl.sum('distance_2d').alias('distance_2d'))
    .with_columns(distance_miles=meters_to_miles(pl.col('distance_2d')))
    .select(['name', 'distance_2d', 'distance_miles'])
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 3))
distances = (tweak_strava(raw)
    .group_by('name')
    .agg(pl.sum('distance_2d').alias('distance_2d'))
    .with_columns(distance_miles=meters_to_miles(pl.col('distance_2d')))
    .select(['name', 'distance_2d', 'distance_miles'])

    .sort('distance_miles')
    .tail(10)
)

ax.barh(distances['name'], distances['distance_miles'])

## Master Aggregation

In [ ]:
# distance per activity

(tweak_strava(raw_lazy)
    .group_by('id')
    .agg(
        activity=pl.col('name').first(),
        distance=pl.col('distance_2d').sum(),
        elevation=pl.col('elevation').sum(),
        elapsed=pl.col('elapsed').last(),
        period=pl.col('period').last(),
    )
    .with_columns(speed=pl.col('distance') / pl.col('elapsed'))
    .sort('distance')
    .collect()
    )

In [ ]:
# filter > 23 hours

print(tweak_strava(raw_lazy)
    .group_by('id')
    .agg(
        activity=pl.col('name').first(),
        distance=pl.col('distance_2d').sum(),
        elevation=pl.col('elevation').sum(),
        elapsed=pl.col('elapsed').last(),
        period=pl.col('period').last(),
    )
    .with_columns(speed=pl.col('distance') / pl.col('elapsed'))
    .sort('elapsed')
    .filter(pl.col('elapsed') < 23 * 60 * 60)
    .collect()
    )

In [ ]:
# plot
(tweak_strava(raw_lazy)
    .group_by('id')
    .agg(
        activity=pl.col('name').first(),
        distance=pl.col('distance_2d').sum(),
        elevation=pl.col('elevation').sum(),
        elapsed=pl.col('elapsed').last(),
        period=pl.col('period').last(),
    )
    .with_columns(speed=pl.col('distance') / pl.col('elapsed'))
    .filter(pl.col('elapsed') < 23 * 60 * 60)
    .sort('distance')
    .tail(20)
    .collect()
    .select([pl.col('activity').cast(pl.Utf8), 'distance', ])
    .sort('distance')
    .plot.bar(y='activity', x='distance')
    )

In [ ]:
import matplotlib.pyplot as plt
import hvplot.polars
import hvplot
hvplot.extension('matplotlib')
fig, ax = plt.subplots(figsize=(6,4), dpi=300)
# plot
(tweak_strava(raw_lazy)
    .group_by('id')
    .agg(
        activity=pl.col('name').first(),
        distance=pl.col('distance_2d').sum(),
        elevation=pl.col('elevation').sum(),
        elapsed=pl.col('elapsed').last(),
        period=pl.col('period').last(),
    )
    .with_columns(speed=pl.col('distance') / pl.col('elapsed'))
    .filter(pl.col('elapsed') < 23 * 60 * 60)
    .sort('distance')
    
    .tail(20)
    .collect()
    .select([pl.col('activity').cast(pl.Utf8), 'distance', ])
    .hvplot.barh(x='activity', y='distance')
    )


In [ ]:
# common aggregation - value counts
with pl.Config(set_tbl_rows=20):
    display(tweak_strava(raw_lazy)
    .select(pl.col('type').value_counts())
    .collect()
    )

In [ ]:
# use unnest to get the values out of the struct
with pl.Config(set_tbl_rows=20):
    display(tweak_strava(raw_lazy)
    .select(pl.col('type').value_counts())
    .collect()
    .unnest('type')
    )

In [ ]:
# how far have I biked each year?
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
  .group_by([pl.col('time').dt.year().alias('year'), 'type'])
  .agg(
      distance=pl.col('distance_2d').sum(),
  )
  .collect()
)


In [ ]:
# pivot type
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
    .group_by([pl.col('time').dt.year().alias('year'), 'type'])
    .agg(
        distance=pl.col('distance_2d').sum(),
    )
    .pivot(index='year', on='type')
    .collect()
)


In [ ]:
# pivot type
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
    .group_by([pl.col('time').dt.year().alias('year'), 'type'])
    .agg(
        distance=pl.col('distance_2d').sum(),
    )
    .collect()
    .pivot(index='year', on='type')
    )


In [ ]:
# sort year (remember that Polars is multi-threaded)
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
    .group_by([pl.col('time').dt.year().alias('year'), 'type'])
    .agg(
        distance=pl.col('distance_2d').sum(),
    )
    .collect()
    .pivot(index='year', on='type')
    .sort('year')
    )


In [ ]:
# plot
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
    .group_by([pl.col('time').dt.year().alias('year'), 'type'])
    .agg(
        distance=pl.col('distance_2d').sum(),
    )
    .collect()
    .pivot(index='year', on='type')
    .sort('year')
    .hvplot.line(x='year', y=['cycling', 'ebikeride'])
    )


In [ ]:
# Do weekly instead of yearly with .group_by_dynamic
# then add a rolling mean
(tweak_strava(raw_lazy)
  .filter(pl.col('type').is_in(['ebikeride', 'cycling']))
  .sort('time')
    #.group_by([pl.col('time').dt.year().alias('year'), 'type'])
    .group_by_dynamic(index_column='time', every='1w', by='type')
    .agg(
        distance=pl.col('distance_2d').sum(),
    )
    .collect()
    .pivot(index='time', on='type')
    # rolling mean
    .with_columns(
        cycling=pl.col('cycling')
        .fill_null(0)
        .rolling_mean(window_size=4),
        ebikeride=pl.col('ebikeride')
        .fill_null(0)
        .rolling_mean(window_size=4),
    )
    .sort('time')
    .hvplot.line(x='time', y=['cycling', 'ebikeride'])
    )


# Exercise: Aggregation

- Create a column *elevation_gained* that calculates the growth of elevation over an activity. (hint: use `.over`)
- What is the average *elevation_gained* for each *year*?
- What is the monthly distance for each *year*?
- What is the monthly distance for each *year* and activity type?

## Conclusion

* Correct types save space and enable convenient math, string, and date functionality
* Chaining operations will:
   * Make code readable
   * Remove bugs
   * Easier to debug
* ``.map_elements`` is slow for math
* Aggregations are powerful. Play with them until they make sense


Let's connect! Reach out (email, LinkedIn) if your team needs help with the Python data stack.

## Pandas Comparison

In [ ]:
import pandas as pd

# Define column lists

def tweak_strava_pd(df: pd.DataFrame) -> pd.DataFrame:
    float_cols = ['distance_2d', 'latitude', 'longitude', 'elevation', 'speed_between']
    cat_cols   = ['name', 'type', 'filename']
    cols       = ['course', 'distance_2d', 'latitude', 'longitude', 'time', 'elevation',
                'speed_between', 'name', 'type', 'filename']
    final_cols = ['distance_2d', 'latitude', 'longitude', 'time', 'elevation', 'speed_between',
                'name', 'type', 'filename', 'id', 'elapsed', 'period']

    return (
        df
        .loc[:, cols]
        .assign(
            # Create 'id' from the filename by splitting, taking the last piece,
            # removing '.gpx', and converting to int64.
            id=lambda d: d['filename']
                .str.split('/')
                .str[-1]
                .str.replace('.gpx', '', regex=False)
                .astype('int64')
        )
        .assign(
            # Cast float_cols to float32.
            **{col: (lambda d, col=col: d[col].astype('float32')) for col in float_cols}
        )
        .assign(
            # Cast cat_cols to categorical dtype.
            **{col: (lambda d, col=col: d[col].astype('category')) for col in cat_cols}
        )
        .assign(
            # Convert the 'time' column to the 'America/Denver' timezone.
            time=lambda d: d['time'].dt.tz_convert('America/Denver')
        )
        .assign(
            # Compute 'elapsed': difference in seconds from the minimum time per 'id'.
            elapsed=lambda d: d.groupby('id')['time']
                               .transform(lambda x: (x - x.min()).dt.total_seconds())
                               .astype('int32')
        )
        .assign(
            # Compute 'period': time difference from the first time in each 'id' group.
            period=lambda d: d.groupby('id')['time']
                              .transform(lambda x: x - x.iloc[0])
        )
        [final_cols]
    )

# Example usage:
raw_pd = pd.read_parquet('data/bikeski.parquet')
tweak_strava_pd(raw_pd)



In [ ]:
%%time
limit = 23 * 60 * 60  # 23 hours in seconds

(tweak_strava_pd(raw_pd)
    .groupby('id')
    .agg(
        activity=('name', 'first'),
        distance=('distance_2d', 'sum'),
        elevation=('elevation', 'sum'),
        elapsed=('elapsed', 'last'),
        period=('period', 'last')
    )
    .assign(speed=lambda df: df['distance'] / df['elapsed'])
    .sort_values('elapsed')
    .query("elapsed < @limit")
)